In [2]:
import pandas
import numpy
from multiprocessing import Pool

#pandas.set_option('display.height', 1000)
#pandas.set_option('display.max_rows', 500)
#pandas.set_option('display.max_columns', 5000)
#pandas.set_option('display.width', 10000)
#pandas.set_option('max_colwidth',100)

In [ ]:
num_partitions = 64 #number of partitions to split dataframe
num_cores = 16 #number of cores on your machine

def parallelize_dataframe(df, func):
    df_split = numpy.array_split(df, num_partitions)
    pool = Pool(num_cores)
    df = pandas.concat(pool.map(func, df_split))
    pool.close()
    pool.join()
    return df

In [ ]:
# Read data:
#microdados = pandas.read_csv("datasets/2017/DADOS/MICRODADOS_ENEM_2017.csv", sep=';', encoding="latin-1", usecols=[0,2, 78, 79,80,81,86,87,88,89,94,95,96,97,98,103,104,105,106,107,108,109])
info_items = pandas.read_csv("datasets/2017/DADOS/ITENS_PROVA_2017.csv", sep=';', encoding="latin-1")

In [ ]:
# Fix question notebook code columns:
microdados['CO_PROVA_CH'].fillna(0, inplace=True)
microdados['CO_PROVA_CH'] = microdados['CO_PROVA_CH'].astype(numpy.int16)
microdados['CO_PROVA_LC'].fillna(0, inplace=True)
microdados['CO_PROVA_LC'] = microdados['CO_PROVA_LC'].astype(numpy.int16)
microdados['CO_PROVA_MT'].fillna(0, inplace=True)
microdados['CO_PROVA_MT'] = microdados['CO_PROVA_MT'].astype(numpy.int16)
microdados['CO_PROVA_CN'].fillna(0, inplace=True)
microdados['CO_PROVA_CN'] = microdados['CO_PROVA_CN'].astype(numpy.int16)
microdados

In [ ]:
new_columns = ["RESPOSTA_"+str(x) for x in info_items['CO_ITEM'].unique()]

In [ ]:
# Create new dataframe with only the answers for each question
new_df = pandas.DataFrame(numpy.empty((microdados.shape[0], len(new_columns)), dtype=numpy.int8), columns=new_columns, dtype=numpy.int8)

In [ ]:
faster = {}
for i in info_items['CO_PROVA'].unique():
    faster[i] = {}
    for j, b in info_items[info_items['CO_PROVA'] == i][['CO_POSICAO','CO_ITEM']].iterrows():
        faster[i][int(b['CO_POSICAO'])] = int(b['CO_ITEM'])

In [ ]:
def fill_answers(df):
    for index,row in df.iterrows():
        name = index
        micdad = microdados.iloc[name][['CO_PROVA_LC','CO_PROVA_MT','CO_PROVA_CN','CO_PROVA_CH', 'TX_RESPOSTAS_LC','TX_RESPOSTAS_MT','TX_RESPOSTAS_CN','TX_RESPOSTAS_CH']]
        tipo_prova = "LC"
        id_prova = micdad['CO_PROVA_' + tipo_prova]
        numq = 50
        #print(tipo_prova)
        #print(int(id_prova)
        if id_prova == 0:
            continue
        #df_prova = info_items[info_items['CO_PROVA'] == int(id_prova)][['CO_POSICAO', 'CO_ITEM']]
        respostas = str(micdad['TX_RESPOSTAS_' + tipo_prova])
        for q in range(0,numq):
            #q_id = df_prova.loc[df_prova['CO_POSICAO'] == q + 1]['CO_ITEM']
            row['RESPOSTA_' + str(faster[id_prova][q + 1])] = ord(respostas[q])
        for tipo_prova in ["CN", "CH", "MT"]:
            id_prova = micdad['CO_PROVA_' + tipo_prova]
            numq = 45
            #print(tipo_prova)
            #print(int(id_prova)
            if id_prova == 0:
                continue
            #df_prova = info_items[info_items['CO_PROVA'] == int(id_prova)][['CO_POSICAO', 'CO_ITEM']]
            respostas = str(micdad['TX_RESPOSTAS_' + tipo_prova])
            for q in range(0,numq):
                #q_id = df_prova.loc[df_prova['CO_POSICAO'] == q + 1]['CO_ITEM']
                row['RESPOSTA_' + str(faster[id_prova][q + 1])] = ord(respostas[q])
    return df

# import cProfile
# import time
# new_df[5000005:500015].to_csv("teste1")
# start = time.time()
# #cProfile.run('fill_answers(new_df.iloc[0:10])')
# fill_answers(new_df.iloc[5000005:5000015])
# fim = time.time()
# print(fim - start)
# new_df[5000005:5000015].to_csv("teste2")
#print(new_df.iloc[400:405].iterrows())
#fill_answers(new_df[new_df['NU_INSCRICAO'] == 170003271004])
#fill_answers(new_df.iloc[1245:1246])
#new_df.iloc[1245]
#even_new_df = new_df.iloc[100:500].copy()
#print(even_new_df)
new_df = parallelize_dataframe(new_df, fill_answers)
#fill_answers(even_new_df)
#print(even_new_df)

In [ ]:
new_df = pandas.read_pickle('real_question_answers.pk')

In [ ]:
answer_counts = {}
answer_ranks = {}
answer_oddness = {}
for i in new_columns:
    answer_counts[i] = new_df[i].value_counts()
    count_array = numpy.array([answer_counts[i].get(65, 0), answer_counts[i].get(66, 0), answer_counts[i].get(67, 0), answer_counts[i].get(68, 0), answer_counts[i].get(69, 0)])
    sorted_count = numpy.sort(count_array)
    answer_counts[i]['total_valid'] = count_array.sum()
    answer_ranks[i] = {}
    answer_ranks[i][65] = numpy.where(sorted_count==answer_counts[i].get(65, 0))[0][0] + 1
    answer_ranks[i][66] = numpy.where(sorted_count==answer_counts[i].get(66, 0))[0][0] + 1
    answer_ranks[i][67] = numpy.where(sorted_count==answer_counts[i].get(67, 0))[0][0] + 1
    answer_ranks[i][68] = numpy.where(sorted_count==answer_counts[i].get(68, 0))[0][0] + 1
    answer_ranks[i][69] = numpy.where(sorted_count==answer_counts[i].get(69, 0))[0][0] + 1
    
    answer_oddness[i] = {}
    answer_oddness[i][0] = 0
    answer_oddness[i][46] = 0
    answer_oddness[i][42] = 0
    answer_oddness[i][57] = 0
    answer_oddness[i][65] = (6 - answer_ranks[i][65])**2 * (1 - (answer_counts[i].get(65, 0))/answer_counts[i]['total_valid'])
    answer_oddness[i][66] = (6 - answer_ranks[i][66])**2 * (1 - (answer_counts[i].get(66, 0))/answer_counts[i]['total_valid'])
    answer_oddness[i][67] = (6 - answer_ranks[i][67])**2 * (1 - (answer_counts[i].get(67, 0))/answer_counts[i]['total_valid'])
    answer_oddness[i][68] = (6 - answer_ranks[i][68])**2 * (1 - (answer_counts[i].get(68, 0))/answer_counts[i]['total_valid'])
    answer_oddness[i][69] = (6 - answer_ranks[i][69])**2 * (1 - (answer_counts[i].get(69, 0))/answer_counts[i]['total_valid'])
    
    
    

In [ ]:
print(answer_counts[new_columns[0]])
print(answer_ranks[new_columns[0]])
print(answer_oddness[new_columns[0]])

In [ ]:
#answer_counts[new_columns[5]]
#answer_ranks[new_columns[5]]

In [ ]:
odd_columns = ["ODDNESS_"+str(x) for x in info_items['CO_ITEM'].unique()]
#kmeans_dataset = pandas.DataFrame(numpy.empty((new_df.shape[0], len(new_columns)), dtype=numpy.float32), columns=odd_columns, dtype=numpy.float32)
kmeans_dataset_2 = new_df.iloc[5000000:-1].copy().astype(numpy.float32)
#kmeans_dataset = kmeans_dataset.rename(odd_columns, copy=0, axis='columns', inplace=True)
#def set_oddness(df):
kmeans_dataset_2.columns = odd_columns
   

In [ ]:
# import time
# start = time.time()
# for i in range(0,100):
#     for j in range(0,553):
#         candidate_answer = new_df.iloc[i][new_columns[j]]
#         kmeans_dataset.iloc[i][odd_columns[j]] = answer_oddness[new_columns[j]][candidate_answer]
# end=time.time()
# print(end-start)

def make_dataset(df):
    for index,row in df.iterrows():
        for j in range(0,553):
            candidate_answer = int(row[odd_columns[j]])
            row[odd_columns[j]] = answer_oddness[new_columns[j]][candidate_answer]
    return df
import time
import cProfile
start = time.time()
#cProfile.run('make_dataset(kmeans_dataset.iloc[0:100])')
kmeans_dataset_2 = parallelize_dataframe(kmeans_dataset_2, make_dataset)
#make_dataset(kmeans_dataset.iloc[1:100])
end=time.time()
print(end-start)


In [ ]:
kmeans_dataset_2.to_pickle("kmeans_dataset_4.pk")

In [ ]:
new_df

In [ ]:
del kmeans_dataset

In [3]:
kmeans_dataset = pandas.concat([
pandas.read_pickle("kmeans_dataset_1.pk"),
pandas.read_pickle("kmeans_dataset_2.pk"),
])


In [4]:
kmeans_dataset

,ODDNESS_48074,ODDNESS_37043,ODDNESS_7463,ODDNESS_54099,ODDNESS_18212,ODDNESS_23183,ODDNESS_33118,ODDNESS_88785,ODDNESS_60767,ODDNESS_47905,...,ODDNESS_12553,ODDNESS_48026,ODDNESS_47561,ODDNESS_39744,ODDNESS_50280,ODDNESS_78667,ODDNESS_55102,ODDNESS_45024,ODDNESS_48116,ODDNESS_42246
0,0.486619,0.486405,0.721620,0.441251,14.241522,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0.000000,0.000000,0.000000,0.000000,0.000000,0.700077,0.560161,0.549008,6.980449,7.410727,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,3.162302,0.486405,2.948041,2.978706,23.563284,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
5,0.000000,0.000000,0.000000,0.000000,0.000000,7.013681,0.560161,0.549008,3.083095,7.410727,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
6,0.000000,0.000000,0.000000,0.000000,0.000000,0.700077,0.560161,7.682171,0.708828,0.681252,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
7,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
8,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
9,7.641050,0.486405,13.376554,23.671825,3.164292,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [ ]:
import time
from sklearn import metrics
from sklearn.cluster import KMeans
from sklearn.datasets import load_digits
from sklearn.decomposition import PCA
from sklearn.preprocessing import scale


print("n_clusters, cost_function, time_taken")
for NClusters in [1,3,5,8,13,21,34,55,89,144,233,377,610,987,1200, 1400,1597, 1700, 2000, 2584]:
    start_time = time.time()
    kmeans = KMeans(init='k-means++', n_clusters=NClusters, n_init=5, n_jobs=16)
    kmeans.fit(kmeans_dataset)
    print(str(NClusters) + ", " + str(kmeans.inertia_) + ", " + str(time.time() - start_time))
    # Obtain labels for each point in mesh. Use last trained model.
    np.savetxt('predict/' + str(NClusters) + 'cluster.out', kmeans.predict(kmeans_dataset), delimiter=',')